# Initialize

In [ ]:
!pip install p3_data openpyxl

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
import seaborn as sns
import json
from io import StringIO
import importlib
import re

In [ ]:
import p3_data
from p3_data import (glob_file_list , load_json_from_file, merge_dicts, plot_groups, 
    get_varying_column_names, filter_dataframe, take_varying_columns,
    load_json_records_as_dataframe, flatten_multiindex_columns)

# Load and Clean Results

In [ ]:
# Load result files from P3 Test Driver
src_files = []
src_files += ['../data/p3_test_driver/results/*rapids*.json.bz2']
raw_df = load_json_records_as_dataframe(src=src_files, ignore_error=True)

In [ ]:
# Clean raw results
def clean_result(result):
    try:
        r = result.copy()
        r['utc_begin'] = pd.to_datetime(r['utc_begin'], utc=True)
        r['utc_end'] = pd.to_datetime(r['utc_end'], utc=True)
        if r['single_batch'] == False:
            r['compute_sec'] = np.sum(r['compute_sec_list'])
            r['input_files_size'] = np.sum(r['input_file_sizes'])
            r['compute_MB_per_sec'] = 1e-6 * r['input_files_size'] / r['compute_sec']
            r['sec_per_batch'] = r['compute_sec'] / r['num_batches']
            r['init_sec'] = r['glob_sec'] + r['getsize_sec'] + r['create_ddf_sec']
        return pd.Series(r)
    except Exception as e:
        print('ERROR: %s: %s' % (r['test_uuid'], e))
        # raise e

In [ ]:
#r = clean_result(raw_df.iloc[-1])
#pd.DataFrame(r)

In [ ]:
clean_df = raw_df.apply(clean_result, axis=1)
clean_df = clean_df.set_index('test_uuid', drop=False)
clean_df = clean_df[clean_df.error==False]
clean_df['compression'] = clean_df['compression'].fillna('None')
clean_df['warmup'] = clean_df['warmup'].fillna(False)
clean_df = clean_df.sort_values(['utc_begin'])

# Explore data

In [ ]:
# Show list of columns
list(clean_df.columns.values)

In [ ]:
# Define columns that identify test parameters
param_cols = [
 'cached',
 'compression',
 'dask_cudf_version',
 'file_format',
 'input_files_size',
 'num_batches',
 'partitions',
 'size_multiplier',
 'stripe_size_MiB',
 'storage_type',
 'test',
 'warmup',
]

In [ ]:
# Define columns that are the output of the experiments
output_cols = [
    'compute_MB_per_sec',
    'compute_sec',
    'init_sec',
    'sec_per_batch',
    'utc_begin',    
]

In [ ]:
cols = param_cols + output_cols

In [ ]:
# View most recent results
clean_df[cols].tail(3).T

In [ ]:
# Export to CSV
clean_df[cols].to_csv('results.csv')

In [ ]:
filt1_df = filter_dataframe(
    clean_df,
    cached=False,
    compression='snappy',
    data_file_prefix = 'perf-no-strings',
    file_format='orc',
    size_multiplier=6.0,
    stripe_size_MiB=2048,
    utc_begin=(pd.Timestamp('2019-12-05 20:25:24', tz='UTC'),
               pd.Timestamp('2019-12-05 21:57:19', tz='UTC')),
    warmup=False,
)
len(filt1_df)

In [ ]:
df = filt1_df.fillna(0).groupby(param_cols).agg({
    'compute_MB_per_sec': ['size','min','mean','max'],
    'compute_sec': ['min','mean','max'],
    'sec_per_batch': ['min','mean','max'],
    'init_sec': ['min','mean','max'],
})
df

In [ ]:
df.unstack(['storage_type'])

In [ ]:
50*48